In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('C:/Users/razieh/Downloads/dataset.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    training_sentences.append(intent['input'])
    training_labels.append(intent['tag'])
    responses.append(intent['output'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [6]:
training_labels
training_sentences
responses
# labels

['A red rose typically symbolizes love, passion, and romance.',
 'Roses prefer well-draining soil, full sun, and regular watering. Prune in the spring to promote new growth and remove dead or diseased branches.',
 'Rose plants can be affected by diseases such as black spot, powdery mildew, and rust. Regularly inspect your plants and treat any issues promptly.',
 'Roses are typically planted in the spring or fall, when the weather is mild and there is plenty of moisture in the soil.',
 'Lilies prefer a well-draining soil and partial shade. Keep the soil moist but not waterlogged, and fertilize everyfew weeks during the growing season.',
 'There are many types of lilies, including Asiatic lilies, Oriental lilies, and trumpet lilies. Each type has its own unique characteristics and growing requirements.',
 'Yes, lilies can be grown in containers as long as the container is large enough and has good drainage. Use a high-quality potting mix and fertilize regularly to promote healthy growth.

In [7]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [8]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [9]:
word_index

{'<OOV>': 1,
 'what': 2,
 'is': 3,
 'the': 4,
 'how': 5,
 'i': 6,
 'of': 7,
 'do': 8,
 'plant': 9,
 'a': 10,
 'my': 11,
 'sunflower': 12,
 'meaning': 13,
 'care': 14,
 'for': 15,
 'rose': 16,
 'are': 17,
 'in': 18,
 'orchid': 19,
 'some': 20,
 'common': 21,
 'diseases': 22,
 'that': 23,
 'affect': 24,
 'plants': 25,
 'should': 26,
 'lily': 27,
 'lilies': 28,
 'can': 29,
 'be': 30,
 'grown': 31,
 'containers': 32,
 'daisy': 33,
 'seeds': 34,
 'red': 35,
 'best': 36,
 'time': 37,
 'year': 38,
 'to': 39,
 'roses': 40,
 'types': 41,
 'there': 42,
 'white': 43,
 'colors': 44,
 'daisies': 45,
 'come': 46,
 'difference': 47,
 'between': 48,
 'and': 49,
 'tallest': 50,
 'on': 51,
 'record': 52,
 'harvest': 53,
 'sunflowers': 54,
 'often': 55,
 'water': 56,
 'orchids': 57,
 'an': 58}

In [8]:
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  4, 13,
         7, 10, 35, 16],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  8,  6, 14,
        15, 11, 16,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2, 17, 20, 21, 22,
        23, 24, 16, 25],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  4, 36, 37,  7,
        38, 39,  9, 40],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5, 26,  6, 14,
        15, 11, 27,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2, 41,
         7, 28, 17, 42],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 29, 28,
        30, 31, 18, 32],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  3,  4, 13,
         7, 10, 43, 27],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2, 44,
         8, 45, 46, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  5,  8,
         6,  9, 33, 34],
       [ 0,  0,  0,  0,  0,  0

In [10]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 128)               2176      
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 18,821
Trainable params: 18,821
Non-trainable params: 0
_________________________________________________________________


In [11]:
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras import regularizers
from keras.optimizers import RMSprop

# Define the model architecture
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = RMSprop(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, batch_size=4, validation_data=(padded_sequences, np.array(training_labels)))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(padded_sequences, np.array(training_labels), verbose=2)
print('Test accuracy:', test_acc)

c:\Users\razieh\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/500
5/5 [==============================] - 2s 104ms/step - loss: 2.2550 - accuracy: 0.1500 - val_loss: 2.2093 - val_accuracy: 0.2000
Epoch 2/500
5/5 [==============================] - 0s 16ms/step - loss: 2.1929 - accuracy: 0.1000 - val_loss: 2.1637 - val_accuracy: 0.2500
Epoch 3/500
5/5 [==============================] - 0s 8ms/step - loss: 2.1507 - accuracy: 0.2000 - val_loss: 2.1267 - val_accuracy: 0.3500
Epoch 4/500
5/5 [==============================] - 0s 8ms/step - loss: 2.1163 - accuracy: 0.2500 - val_loss: 2.0936 - val_accuracy: 0.3500
Epoch 5/500
5/5 [==============================] - 0s 10ms/step - loss: 2.0810 - accuracy: 0.2000 - val_loss: 2.0629 - val_accuracy: 0.2500
Epoch 6/500
5/5 [==============================] - 0s 8ms/step - loss: 2.0539 - accuracy: 0.3000 - val_loss: 2.0347 - val_accuracy: 0.3500
Epoch 7/500
5/5 [==============================] - 0s 8ms/step - loss: 2.0265 - accuracy: 0.3000 - val_loss: 2.0079 - val_accuracy: 0.2500
Epoch 8/500
5/5 [======

In [12]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [15]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import random
import pickle

with open("C:/Users/razieh/Downloads/dataset.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        print(result)

        for i in data['intents']:
            if i['tag'] == tag:
                print(i['output'])

chat()

1/1 [==============================] - 0s 59ms/step
[[0.001712   0.0007062  0.6942973  0.09092472 0.21235973]]
Orchids generally need to be watered once a week, and should be allowed to dry out slightly between waterings. Be careful not to overwater, as this can cause root rot.
Orchids require specific care, including bright but indirect light, well-draining soil, and regular fertilization. Research the specific care requirements for your orchid species.
Orchids can beaffected by diseases such as root rot, leaf spot, and fungal infections. Proper care, including good air circulation and avoiding overwatering, can help prevent these issues.
Orchids are often associated with beauty, love, and strength, and are often given as gifts to show admiration or appreciation.
